In [1]:
!pip install keras_unet
!pip install nibabel
!pip install focal-loss

In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold 
from keras_unet.models import custom_unet
import keras
import tensorflow as tf 
from tensorflow.keras.optimizers import Adam, SGD
from keras_unet.metrics import iou, iou_thresholded, dice_coef
#from google.colab import drive

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [3]:
#drive.mount('/content/drive')

In [4]:
#os.chdir('drive/My Drive/ML4HC Task1')

In [5]:
from img_generator import build_train_test_df, DataGenerator2D, DataGenerator3D
from model_utils import calculate_iou_holdout_set, jaccard_distance_loss, focal_loss, dice_coef_loss, binary_focal_loss

## Create dataframes in the format and with the information required by the generator

### Create datframes with paths and depth for the images

In [6]:
data_path_source_dir = os.path.join('ml4h_proj1_colon_cancer_ct', 'ml4h_proj1_colon_cancer_ct')

In [7]:
tr_df, x_ts_df = build_train_test_df(data_path_source_dir)

In [8]:
tr_3d_df = tr_df.reset_index(level=1).drop_duplicates(['x_tr_img_path', 'y_tr_img_path'], keep='last')\
                .loc[:, ['x_tr_img_path', 'y_tr_img_path', 'depth']]

x_ts_3d_df = x_ts_df.reset_index(level=1).drop_duplicates(['x_ts_img_path'], keep='last')\
                .loc[:, ['x_ts_img_path', 'depth']]

In [9]:
tr_3d_df.head(4)

,x_tr_img_path,y_tr_img_path,depth
index,,,
001,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,60
005,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,98
006,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,68
007,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,74


As we are going to read 3d image by 3d image, let's modify the dataframes so that they are not indexed by the number of cuts

In [10]:
x_ts_3d_df.head(4)

,x_ts_img_path,depth
index,,
171,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,119
172,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,81
175,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,90
176,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,113


### Create CV folds for `tr_df`

let's go for 10 folds to have a 90/10 split. We can still only use only 3 or 5 to estimate the metrics

In [11]:
def generate_fold_dict(df_, n_folds=3, seed=123):
    
    img_num_idx_list = df_.index
    folder = KFold(n_splits=n_folds, random_state=seed, shuffle=True)
    df_fold_dict = dict()
    
    for i, (train_fold_i, holdout_i) in enumerate(folder.split(img_num_idx_list)):
        train_fold_i_idx = img_num_idx_list[train_fold_i]
        holdout_i_idx = img_num_idx_list[holdout_i]

        df_fold_dict[f'fold_{i}'] = {
            'train': df_.loc[train_fold_i_idx, :],
            'holdout': df_.loc[holdout_i_idx, :]
        }
        
    return df_fold_dict

In [12]:
tr_fold_df_dict =  generate_fold_dict(df_=tr_3d_df, n_folds=10, seed=123) # 10 folds to have 90/10 split, but we can use only 3 or 5 to estimate the metrics

In [13]:
tr_fold_df_dict['fold_0']['train']

,x_tr_img_path,y_tr_img_path,depth
index,,,
005,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,98
006,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,68
007,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,74
011,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,90
012,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,88
...,...,...,...
164,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,90
165,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,142
166,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,87


In [14]:
tr_fold_df_dict['fold_0']['holdout']

,x_tr_img_path,y_tr_img_path,depth
index,,,
001,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,60
008,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,149
009,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,90
015,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,54
050,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,142
096,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,91
115,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,97
126,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,85
141,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,ml4h_proj1_colon_cancer_ct\ml4h_proj1_colon_ca...,90


## Let's create a generator for the trainset

For the first fold

In [15]:
tr_fold_0_df = tr_fold_df_dict['fold_0']['train']
holdout_fold_0_df = tr_fold_df_dict['fold_0']['holdout']
resize_dim = (128, 128)
print(tr_fold_0_df.shape[0])
print(holdout_fold_0_df.shape[0])

90
10


In [16]:
train_data_generator = DataGenerator3D(
    df=tr_fold_0_df, x_col='x_tr_img_path', y_col='y_tr_img_path',
    batch_size=1, shuffle=False, resize_dim=resize_dim, 
    hounsfield_min= -1000., hounsfield_max=400.,
    rotate_range=30, horizontal_flip=True, vertical_flip=True,
    random_crop=(0.8, 0.9)) #,
    #shearing=((0.1, 0.3), (0., 0.0)), gaussian_blur=(0.3162, 0.9487))

holdout_data_generator = DataGenerator3D(
    df=holdout_fold_0_df, x_col='x_tr_img_path', y_col='y_tr_img_path',
    batch_size=1, shuffle=False, resize_dim=resize_dim,
    hounsfield_min= -1000., hounsfield_max=400.)

In [17]:
num_epoch = 5
version = 1
split = '90_10_split'
loss_used = 'dice_loss'
augmentations = 'flips_rot_crop'
depth_shuffle = 'depth_shuffle'
imbalance_sampling = '  '
attepmt_name_dir = f'v{version}_{num_epoch}epochs_{split}_{loss_used}_{augmentations}_{depth_shuffle}_{imbalance_sampling}' 
attepmt_name_dir = os.path.join('training_runs', 'juan', '3dUnet', attepmt_name_dir)
os.makedirs(attepmt_name_dir, exist_ok=True)

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv3D, Input, MaxPooling3D, Dropout, concatenate, UpSampling3D
import tensorflow as tf

def Unet3D(inputs, num_classes):
    x=inputs
    conv1 = Conv3D(8, 3, activation = 'relu', padding = 'same',data_format="channels_last")(x)
    conv1 = Conv3D(8, 3, activation = 'relu', padding = 'same')(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(conv1)
    conv2 = Conv3D(16, 3, activation = 'relu', padding = 'same')(pool1)
    conv2 = Conv3D(16, 3, activation = 'relu', padding = 'same')(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(conv2)
    conv3 = Conv3D(32, 3, activation = 'relu', padding = 'same')(pool2)
    conv3 = Conv3D(32, 3, activation = 'relu', padding = 'same')(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(conv3)
    conv4 = Conv3D(64, 3, activation = 'relu', padding = 'same')(pool3)
    conv4 = Conv3D(64, 3, activation = 'relu', padding = 'same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(drop4)

    conv5 = Conv3D(128, 3, activation = 'relu', padding = 'same')(pool4)
    conv5 = Conv3D(128, 3, activation = 'relu', padding = 'same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv3D(64, 2, activation = 'relu', padding = 'same')(UpSampling3D(size = (2,2,2))(drop5))
    merge6 = concatenate([drop4,up6],axis=-1)
    conv6 = Conv3D(64, 3, activation = 'relu', padding = 'same')(merge6)
    conv6 = Conv3D(64, 3, activation = 'relu', padding = 'same')(conv6)

    up7 = Conv3D(32, 2, activation = 'relu', padding = 'same')(UpSampling3D(size = (2,2,2))(conv6))
    merge7 = concatenate([conv3,up7],axis=-1)
    conv7 = Conv3D(32, 3, activation = 'relu', padding = 'same')(merge7)
    conv7 = Conv3D(32, 3, activation = 'relu', padding = 'same')(conv7)

    up8 = Conv3D(16, 2, activation = 'relu', padding = 'same')(UpSampling3D(size = (2,2,2))(conv7))
    merge8 = concatenate([conv2,up8],axis=-1)
    conv8 = Conv3D(16, 3, activation = 'relu', padding = 'same')(merge8)
    conv8 = Conv3D(16, 3, activation = 'relu', padding = 'same')(conv8)

    up9 = Conv3D(8, 2, activation = 'relu', padding = 'same')(UpSampling3D(size = (2,2,2))(conv8))
    merge9 = concatenate([conv1,up9],axis=-1)
    conv9 = Conv3D(8, 3, activation = 'relu', padding = 'same')(merge9)
    conv9 = Conv3D(8, 3, activation = 'relu', padding = 'same')(conv9)
    conv10 = Conv3D(1, 1, activation = 'sigmoid')(conv9)
    model = Model(inputs=inputs, outputs = conv10)
    #model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [19]:
def unet3d(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, strides=(1, 1, 2),
         image_height=256, image_width=256, image_depth=None, in_channels=1, out_channels=1):
    
    inputs = keras.layers.Input(shape=(image_height, image_width, image_depth, in_channels))
    x = inputs
    
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv3D(initial_features * 2 ** level, **convpars)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool3D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv3DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv3D(initial_features * 2 ** level, **convpars)(x)
            
    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv3D(out_channels, kernel_size=1, activation=activation, padding='same')(x)
    
    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET-L{n_levels}-F{initial_features}')

In [55]:
model_3D = unet3d(n_levels=4, initial_features=32, n_blocks=2, kernel_size=(3,3,3), pooling_size=2, strides=1,
                image_height=128, image_width=128, image_depth=None, in_channels=1, out_channels=1)

In [21]:
model_3D = Unet3D(inputs=keras.layers.Input(shape=(128, 128, None, 1)),
                  num_classes=1)

In [22]:
model_3D.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, No 0                                            
__________________________________________________________________________________________________
conv3d_15 (Conv3D)              (None, 128, 128, Non 224         input_2[0][0]                    
__________________________________________________________________________________________________
conv3d_16 (Conv3D)              (None, 128, 128, Non 1736        conv3d_15[0][0]                  
__________________________________________________________________________________________________
max_pooling3d_3 (MaxPooling3D)  (None, 64, 64, None, 0           conv3d_16[0][0]                  
______________________________________________________________________________________________

In [23]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=20),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(f'{attepmt_name_dir}', 'model_sampling.{epoch:02d}-{val_loss:.2f}.h5')),
    #tf.keras.callbacks.TensorBoard(log_dir=os.path.join(f'{attepmt_name_dir}', 'logs_new')),
]

In [57]:
model_3D.compile(
    optimizer=Adam(learning_rate=0.001), 
    #optimizer=SGD(lr=0.01, momentum=0.99),
    #loss='binary_crossentropy',
    loss=dice_coef_loss,
    metrics=[iou, iou_thresholded]
)

In [73]:
train_data_generator_1 = DataGenerator3D(
    df=tr_fold_0_df.loc['005':'005'], x_col='x_tr_img_path', y_col='y_tr_img_path',
    batch_size=1, shuffle=False, resize_dim=(128, 128, 128), 
    hounsfield_min= -1000., hounsfield_max=400.,
    rotate_range=30, horizontal_flip=True, vertical_flip=True,
    random_crop=(0.8, 0.9)) #,
    #shearing=((0.1, 0.3), (0., 0.0)), gaussian_blur=(0.3162, 0.9487))

train_data_generator_2 = DataGenerator3D(
    df=tr_fold_0_df.loc['166':'166'], x_col='x_tr_img_path', y_col='y_tr_img_path',
    batch_size=1, shuffle=False, resize_dim=resize_dim, 
    hounsfield_min= -1000., hounsfield_max=400.,
    rotate_range=30, horizontal_flip=True, vertical_flip=True,
    random_crop=(0.8, 0.9)) #,
    #shearing=((0.1, 0.3), (0., 0.0)), gaussian_blur=(0.3162, 0.9487))

In [74]:
for i, (X, y) in enumerate(train_data_generator_1):
    print(X.shape)
    print(y.shape)
    assert X.shape == y.shape
    
    break

TypeError: function takes exactly 2 arguments (3 given)

In [48]:
for i, (X, y) in enumerate(train_data_generator_2):
    print(X.shape)
    print(y.shape)
    assert X.shape == y.shape
    
    break

(1, 128, 128, 87, 1)
(1, 128, 128, 87, 1)


In [59]:
model_3D = unet3d(n_levels=4, initial_features=32, n_blocks=2, kernel_size=(3,3,3), pooling_size=2, strides=1,
                image_height=128, image_width=128, image_depth=None, in_channels=1, out_channels=1)

In [60]:
model_3D.compile(
    optimizer=Adam(learning_rate=0.001), 
    #optimizer=SGD(lr=0.01, momentum=0.99),
    #loss='binary_crossentropy',
    loss=dice_coef_loss,
    metrics=[iou, iou_thresholded]
)

In [61]:
model_3D.fit(train_data_generator_1, validation_data=holdout_data_generator,
             epochs=num_epoch, callbacks=my_callbacks)

Epoch 1/5


InvalidArgumentError:  ConcatOp : Dimensions of inputs should match: shape[0] = [1,64,64,48,64] vs. shape[1] = [1,64,64,49,64]
	 [[node UNET-L4-F32/concatenate_14/concat (defined at <ipython-input-61-1f9e2dbe7e85>:2) ]] [Op:__inference_train_function_14285]

Function call stack:
train_function


In [62]:
model_3D.fit(train_data_generator_2, validation_data=holdout_data_generator,
             epochs=num_epoch, callbacks=my_callbacks)

Epoch 1/5


InvalidArgumentError:  All dimensions except 4 must match. Input 1 has shape [1 32 32 21 128] and doesn't match input 0 with shape [1 32 32 20 128].
	 [[node gradient_tape/UNET-L4-F32/concatenate_13/ConcatOffset (defined at <ipython-input-61-1f9e2dbe7e85>:2) ]] [Op:__inference_train_function_14285]

Function call stack:
train_function


In [63]:
model_3D = unet3d(n_levels=4, initial_features=32, n_blocks=2, kernel_size=(3,3,2), pooling_size=2, strides=1,
                image_height=128, image_width=128, image_depth=None, in_channels=1, out_channels=1)

In [64]:
model_3D.compile(
    optimizer=Adam(learning_rate=0.001), 
    #optimizer=SGD(lr=0.01, momentum=0.99),
    #loss='binary_crossentropy',
    loss=dice_coef_loss,
    metrics=[iou, iou_thresholded]
)

In [65]:
model_3D.fit(train_data_generator_1, validation_data=holdout_data_generator,
             epochs=num_epoch, callbacks=my_callbacks)

Epoch 1/5


InvalidArgumentError:  ConcatOp : Dimensions of inputs should match: shape[0] = [1,64,64,48,64] vs. shape[1] = [1,64,64,49,64]
	 [[node UNET-L4-F32/concatenate_17/concat (defined at <ipython-input-65-1f9e2dbe7e85>:2) ]] [Op:__inference_train_function_16750]

Function call stack:
train_function


In [66]:
model_3D.fit(train_data_generator_2, validation_data=holdout_data_generator,
             epochs=num_epoch, callbacks=my_callbacks)

Epoch 1/5


InvalidArgumentError:  ConcatOp : Dimensions of inputs should match: shape[0] = [1,32,32,20,128] vs. shape[1] = [1,32,32,21,128]
	 [[node UNET-L4-F32/concatenate_16/concat (defined at <ipython-input-65-1f9e2dbe7e85>:2) ]] [Op:__inference_train_function_16750]

Function call stack:
train_function


In [ ]:
inputs = keras.layers.Input(shape=(image_height, image_width, image_depth, in_channels))
x = inputs
    
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv3D(initial_features * 2 ** level, **convpars)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool3D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv3DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv3D(initial_features * 2 ** level, **convpars)(x)
            
    # output
    activation = 'sigmoid' if out_channels == 1 else 'softmax'
    x = keras.layers.Conv3D(out_channels, kernel_size=1, activation=activation, padding='same')(x)

In [72]:
input_shape = (1, 128, 128, 98, 1)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv3D(32, 3, activation='relu', padding='same')(x)
y_pool = keras.layers.MaxPool3D(2)(x)
print(y.shape)
print(y_pool.shape)

(1, 128, 128, 98, 32)
(1, 64, 64, 49, 1)


In [ ]:
model_3D.save(f'./{attepmt_name_dir}/end_of_training_version')

Let's have it predict on the holdout data

In [ ]:
iou_df = calculate_iou_holdout_set(holdout_df_=holdout_fold_0_df, img_dims=resize_dim,
                                   model_=model, pixel_threshold=0.5,
                                   prediction_batch_size=30)

In [ ]:
iou_df

In [ ]:
iou_df.iou.mean()

In [ ]:
a= np.ones((2, 128, 128, 95))

In [ ]:
    def pad_along_axis(array: np.ndarray, target_length: int, axis: int = 0):
        pad_size = target_length - array.shape[axis]

        if pad_size <= 0:
            return array

        npad = [(0, 0)] * array.ndim
        npad[axis] = (0, pad_size)

        return np.pad(array, pad_width=npad, mode='constant', constant_values=0)

In [ ]:
pad_along_axis(a, 96, axis=3)

# Check how a given trained model predicts

In [ ]:
cancer_pixels_df = pd.read_pickle('cancer_pixels_df')
cancer_pixels_df.reset_index(inplace=True)
cancer_pixels_df['index'] = cancer_pixels_df.image_name.map(lambda str_: str_.split('.nii.gz')[0].split('colon_')[1])

tr_df_cancer_info = tr_df.join(
    cancer_pixels_df.set_index(['index', 'depth_i'])[['cancer_pixel_area']],
    how='left')


In [ ]:
img_with_cancer_gen = DataGenerator2D(df=tr_df_cancer_info[~(tr_df_cancer_info.cancer_pixel_area.isna())].sample(20),
                                      x_col='x_tr_img_path', y_col='y_tr_img_path', batch_size=4, num_classes=None, shuffle=False, resize_dim=resize_dim)

img_without_cancer_gen = DataGenerator2D(df=tr_df_cancer_info[tr_df_cancer_info.cancer_pixel_area.isna()].sample(20),
                                     x_col='x_tr_img_path', y_col='y_tr_img_path', batch_size=4, num_classes=None, shuffle=False, resize_dim=resize_dim)

In [ ]:
model = tf.keras.models.load_model(
    f'./{attepmt_name_dir}/model_sampling.01-2.20.h5',
    custom_objects={'iou':iou, 'iou_thresholded': iou_thresholded,
                    'jaccard_distance_loss': jaccard_distance_loss,
                    'binary_focal_loss_fixed': binary_focal_loss(gamma=2., alpha=0.7)})

In [ ]:
#model = tf.keras.models.load_model('./5_epochs_v2', custom_objects={'iou':iou, 'iou_thresholded': iou_thresholded})

In [ ]:
# Let's see how it predicts for images of cancer
for i, (X, y) in enumerate((img_with_cancer_gen)):
    print(f'X: {X.shape}')
    print(f'y: {y.shape}')

    y_pred = model.predict(X)
    print(y_pred.shape)
    
    for i in range(X.shape[0]):
        f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
        f.set_size_inches(20,20)

        ax1.imshow(X[i,:,:])
        ax1.set_title('Input image')
        
        ax2.imshow(y[i,:,:])
        ax2.set_title('Ground truth, target label')

        ax3.imshow(np.squeeze(y_pred[i,:,:]))
        ax3.set_title('Predicted by the model')
        
        plt.show()
        plt.close()

In [ ]:
# Let's see how it predicts for images of cancer
for i, (X, y) in enumerate((img_without_cancer_gen)):
    print(f'X: {X.shape}')
    print(f'y: {y.shape}')

    y_pred = model.predict(X)
    print(y_pred.shape)
    
    for i in range(X.shape[0]):
        f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True)
        f.set_size_inches(20,20)

        ax1.imshow(X[i,:,:])
        ax1.set_title('Input image')
        
        ax2.imshow(y[i,:,:])
        ax2.set_title('Ground truth, target label')

        ax3.imshow(np.squeeze(y_pred[i,:,:]))
        ax3.set_title('Predicted by the model')
        
        plt.show()
        plt.close()
    

# Unet 3D

In [75]:
!pip install sympy

In [1]:
import sympy

In [2]:
sympy.ntheory.factorint(98)

{2: 1, 7: 2}

In [3]:
biggest_3d_depth = 32

In [2]:
90-32

58

In [3]:
58-32

26

In [63]:
def extract_cut(img_depth, biggest_3d_depth):
    diff = img_depth - biggest_3d_depth
    
    if biggest_3d_depth == 2 or img_depth == 1:
        return 0
    
    elif diff > 0:
        print(biggest_3d_depth)
        print(f'diff: {diff}')
        return extract_cut(diff, biggest_3d_depth)
    elif diff == 0 and biggest_3d_depth != 2:
        print(biggest_3d_depth)
        return 0
    
    elif diff < 0:
        print(f'img_depth: {img_depth}')
        print(f'biggest_3d_depth/2: {biggest_3d_depth/2}')
        return extract_cut(img_depth, biggest_3d_depth/2) 
    
    else: 
        return 1

In [64]:
extract_cut(img_depth=101, biggest_3d_depth=32)

32
diff: 69
32
diff: 37
32
diff: 5
img_depth: 5
biggest_3d_depth/2: 16.0
img_depth: 5
biggest_3d_depth/2: 8.0
img_depth: 5
biggest_3d_depth/2: 4.0
4.0
diff: 1.0


0

In [42]:
32 *2 + 16 + 8 + 4

92

In [12]:
32*2 + 16 + 8 + 2

90

In [ ]:
def find_depth_cuts(img_depth, biggest_3d_depth):
    

In [81]:
98/2

49.0